In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains



- captar listado de URLs de los vinos. 


In [2]:
driver = webdriver.Chrome()
print("Abriendo navegador...")
driver.implicitly_wait(10)
driver.maximize_window()
print("Ventana maximizada")
driver.implicitly_wait(10)
driver.get("https://www.vinissimus.com/es/")
print("Ingresando a Vinissimus")
driver.implicitly_wait(10)
driver.implicitly_wait(5)
# Aceptamos las cookies
try:
    driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/button[2]").click()
    print("Cookies aceptadas")
    driver.implicitly_wait(10)
except NoSuchElementException:
    print("No se encontró el botón de cookies")
    driver.implicitly_wait(10)

# Accedemos a Vinos 
driver.find_element(By.XPATH, "/html/body/nav/ul/li[1]/a").click()
print("Accediendo a Vinos")
driver.implicitly_wait(10)

# Accedemos a españoles
driver.find_element(By.XPATH, "/html/body/div/main/div/section/div[1]/button[2]").click()
print("Accediendo a vinos españoles")
driver.implicitly_wait(10)

dict_urls = {"urls": {}}
max_attempts = 3  # Número máximo de intentos para encontrar el botón de la página siguiente

for page in range(7, 15):  # ITERACIÓN DE PÁGINAS
    attempts = 0  # Inicializa el contador de intentos para esta página
    found_button = False  # Bandera para controlar si se encontró el botón de la página
    driver.implicitly_wait(10)
    
    while attempts < max_attempts:
        # Código para cerrar el popup si está presente
        try:
            close_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/h4/span')
            close_button.click()
            print("Popup cerrado")
            driver.implicitly_wait(5)  # Espera corta después de cerrar el popup
        except NoSuchElementException:
            pass  # No se encontró el botón de cierre del popup, continúa con la extracción de URLs
        try:
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/section/nav/button[{page}]').click()
            #try except dentro del try except 4/5 en lugar de page 
            print(f"Clic en la página {page}")
            driver.implicitly_wait(10)
            found_button = True  # Se encontró el botón
            break  # Sale del bucle while si se encuentra el botón
        except NoSuchElementException:
            attempts += 1
            print(f"Intento {attempts}: Botón de la página {page} no encontrado.")
            driver.implicitly_wait(10)

    if not found_button:
        print(f"Botón de la página {page} no encontrado después de {max_attempts} intentos. Pasando a la siguiente página.")
        driver.implicitly_wait(10)
        continue  # Continúa con la siguiente página si el botón no se encuentra después de los intentos especificados

    # Obtiene los datos de URL de vinos 
    enlaces_vinos = driver.find_elements(By.XPATH, '//div[contains(@class, "details")]/a')
    driver.implicitly_wait(10)

    for index, enlace in enumerate(enlaces_vinos, start=1):
        try:
            driver.implicitly_wait(10)
            wine_url = enlace.get_attribute('href')
            dict_urls['urls'][f"Page_{page}_Wine_{index}"] = wine_url
            print(f"URL del vino en la página {page} - Wine_{index}: {wine_url}")
        except NoSuchElementException:
            driver.implicitly_wait(10)
            print(f"No se pudo obtener la URL para el enlace {index} en la página {page}")

driver.quit()
driver.implicitly_wait(10)
print(dict_urls)


Abriendo navegador...
Ventana maximizada
Ingresando a Vinissimus
Cookies aceptadas
Accediendo a Vinos
Accediendo a vinos españoles
Clic en la página 7
URL del vino en la página 7 - Wine_1: https://www.vinissimus.com/es/vino/pruno/
URL del vino en la página 7 - Wine_2: https://www.vinissimus.com/es/cava/raventos-i-blanc-de-nit/
URL del vino en la página 7 - Wine_3: https://www.vinissimus.com/es/cava/vall-dolina-reserva-brut-nature/
URL del vino en la página 7 - Wine_4: https://www.vinissimus.com/es/vino/mauro/
URL del vino en la página 7 - Wine_5: https://www.vinissimus.com/es/vino/emilio-moro-finca-resalso/
URL del vino en la página 7 - Wine_6: https://www.vinissimus.com/es/vino/camins-del-priorat/
URL del vino en la página 7 - Wine_7: https://www.vinissimus.com/es/vino/dominio-del-aguila-reserva/
URL del vino en la página 7 - Wine_8: https://www.vinissimus.com/es/vino/tomas-postigo-tinto-crianza/
URL del vino en la página 7 - Wine_9: https://www.vinissimus.com/es/vino/matarromera-cria

MaxRetryError: HTTPConnectionPool(host='localhost', port=59484): Max retries exceeded with url: /session/7d66a583cadce647f020bf228149478a/timeouts (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1151d75d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
//*[@id="__next"]/main/div[2]/section/nav/button[5]
//*[@id="__next"]/main/div[2]/section/nav/button[4]
//*[@id="__next"]/main/div[2]/section/nav/button[4]
//*[@id="__next"]/main/div[2]/section/nav/button[5]

//*[@id="__next"]/main/div[2]/section/nav/button[4]
//*[@id="__next"]/main/div[2]/section/nav/button[4]

-- llevarme el listado de URL a un punto PY y hacerlo como una query con el listado de las URLs y así llamamos a la función 

-- de ahí, hacemos Bsoup y sacamos los datos por html 